In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'o3': 'atmpro'}
band = [7]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.6, 1.75), (1.55, 1.66, 1.7, 1.75, 1.8)]
  wgt = [(0.35, 0.6), (0.5, 0.55, 0.7, 0.9, 1.0)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.037105                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.029537
0.000750      2.0 -0.041552                  2.0 -0.033122
0.001052      3.0 -0.050101                  3.0 -0.040093
0.001476      4.0 -0.058209                  4.0 -0.046756
0.002070      5.0 -0.065878                  5.0 -0.053119
0.002904      6.0 -0.073098                  6.0 -0.059172
0.004074      7.0 -0.079859                  7.0 -0.064914
0.005714      8.0 -0.086149                  8.0 -0.070318
0.008015      9.0 -0.091964                  9.0 -0.075390
0.011243     10.0 -0.097409                 10.0 -0.080129
0.015771     11.0 -0.102566                 11.0 -0.084533
0.022122     12.0 -0.107518                 12.0 -0.088610
0.031031     13.0 -0.112430                 13.0 -0.092385
0.043528     14.0 -0.116384                 14.0 -0.095106
0.061057     15.0 -0.113203                 15.0 -0.093247
0.085645     16.0 -0.081782                 16.0 -0.070816
0.120136     17.0 -0.000743                 17.0 -0.008119
0.168516     18.0  0.137203                 18.0  0.105224
0.236378     19.0  0.341926                 19.0  0.281839
0.331549     20.0  0.624160                 20.0  0.535796
0.465100     21.0  0.997748                 21.0  0.881753
0.652400     22.0  1.433943                 22.0  1.291870
0.915100     23.0  1.706918                 23.0  1.588076
1.283650     24.0  1.694924                 24.0  1.641716
1.800600     25.0  1.528004                 25.0  1.502718
2.525700     26.0  1.215498                 26.0  1.177079
3.542800     27.0  0.866032                 27.0  0.826540
4.969550     28.0  0.582898                 28.0  0.560609
6.970850     29.0  0.377470                 29.0  0.378793
9.778100     30.0  0.181026                 30.0  0.199856
13.715850    31.0 -0.054924                 31.0 -0.039076
19.239350    32.0 -0.275849                 32.0 -0.275298
26.987250    33.0 -0.454722                 33.0 -0.476179
37.855300    34.0 -0.549165                 34.0 -0.603113
53.100050    35.0 -0.516076                 35.0 -0.585608
73.887500    36.0 -0.346409                 36.0 -0.381741
97.662500    37.0 -0.173220                 37.0 -0.180835
121.437500   38.0 -0.109169                 38.0 -0.110838
145.212500   39.0 -0.089809                 39.0 -0.090275
168.987500   40.0 -0.075865                 40.0 -0.075545
192.762500   41.0 -0.064826                 41.0 -0.063990
216.537500   42.0 -0.054471                 42.0 -0.053350
240.312500   43.0 -0.045550                 43.0 -0.044344
264.087500   44.0 -0.038042                 44.0 -0.036869
287.862500   45.0 -0.031865                 45.0 -0.030788
311.637500   46.0 -0.026852                 46.0 -0.025903
335.412500   47.0 -0.022516                 47.0 -0.021732
359.187500   48.0 -0.018523                 48.0 -0.017936
382.962500   49.0 -0.014893                 49.0 -0.014517
406.737500   50.0 -0.011554                 50.0 -0.011399
430.512500   51.0 -0.008263                 51.0 -0.008340
454.287500   52.0 -0.004928                 52.0 -0.005236
478.062500   53.0 -0.001504                 53.0 -0.002036
501.837500   54.0  0.002040                 54.0  0.001305
525.612500   55.0  0.005752                 55.0  0.004786
549.387500   56.0  0.009663                 56.0  0.008468
573.162500   57.0  0.013815                 57.0  0.012381
596.937500   58.0  0.018255                 58.0  0.016561
620.712500   59.0  0.023001                 59.0  0.021027
644.487500   60.0  0.028159                 60.0  0.025846
668.262500   61.0  0.033714                 61.0  0.031052
692.037500   62.0  0.039393                 62.0  0.036510
715.812500   63.0  0.044915                 63.0  0.042059
739.587500   64.0  0.052117                 64.0  0.049592
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -26.517507  0.000000 -26.517507        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -26.271640  0.000002   
6.244000e-04 2     -26.517511  0.000002 -26.517509 -26.271643  0.000003   
8.759000e-04 3     -26.517513  0.000002 -26.517511 -26.271645  0.000004   
1.228600e-03 4     -26.517516  0.000004 -26.517513 -26.271647  0.000004   
1.723400e-03 5     -26.517522  0.000006 -26.517516 -26.271652  0.000006   
2.417400e-03 6     -26.517531  0.000010 -26.517522 -26.271659  0.000009   
3.390900e-03 7     -26.517546  0.000016 -26.517530 -26.271670  0.000013   
4.756500e-03 8     -26.517568  0.000025 -26.517543 -26.271687  0.000020   
6.672000e-03 9     -26.517603  0.000041 -26.517563 -26.271714  0.000031   
9.358900e-03 10    -26.517657  0.000065 -26.517592 -26.271755  0.000048   
1.312780e-02 11    -26.517738  0.000103 -26.517635 -26.271818  0.000075   
1.841450e-02 12    -26.517860  0.000161 -26.517699 -26.271912  0.000116   
2.583020e-02 13    -26.518044  0.000251 -26.517794 -26.272054  0.000180   
3.623230e-02 14    -26.518320  0.000387 -26.517932 -26.272266  0.000279   
5.082340e-02 15    -26.518730  0.000596 -26.518134 -26.272582  0.000430   
7.129060e-02 16    -26.519368  0.000960 -26.518408 -26.273074  0.000696   
1.000000e-01 17    -26.520400  0.001714 -26.518686 -26.273874  0.001255   
1.402710e-01 18    -26.521978  0.003288 -26.518690 -26.275105  0.002447   
1.967600e-01 19    -26.524197  0.006425 -26.517772 -26.276834  0.004881   
2.759970e-01 20    -26.526993  0.012431 -26.514562 -26.278988  0.009680   
3.871000e-01 21    -26.529863  0.023515 -26.506348 -26.281099  0.018844   
5.431000e-01 22    -26.531219  0.043309 -26.487910 -26.281783  0.035824   
7.617000e-01 23    -26.527495  0.076716 -26.450779 -26.278109  0.065606   
1.068500e+00 24    -26.516394  0.127650 -26.388745 -26.268134  0.113351   
1.498800e+00 25    -26.498277  0.195926 -26.302351 -26.252052  0.180960   
2.102400e+00 26    -26.472550  0.279453 -26.193097 -26.229526  0.265890   
2.949000e+00 27    -26.443251  0.372050 -26.071200 -26.204616  0.359036   
4.136600e+00 28    -26.417796  0.468429 -25.949367 -26.184745  0.455455   
5.802500e+00 29    -26.405071  0.570732 -25.834339 -26.178938  0.560289   
8.139200e+00 30    -26.419916  0.690060 -25.729856 -26.202535  0.688746   
1.141700e+01 31    -26.495310  0.835742 -25.659567 -26.288999  0.852817   
1.601470e+01 32    -26.679386  0.989906 -25.689481 -26.486280  1.028814   
2.246400e+01 33    -27.024953  1.124734 -25.900220 -26.847408  1.179603   
3.151050e+01 34    -27.614193  1.226684 -26.387510 -27.467060  1.288918   
4.420010e+01 35    -28.493953  1.280953 -27.213000 -28.427862  1.343043   
6.200000e+01 36    -29.573497  1.272337 -28.301160 -29.652662  1.332945   
8.577500e+01 37    -30.512524  1.235765 -29.276759 -30.692078  1.297147   
1.095500e+02 38    -30.982635  1.218033 -29.764602 -31.184032  1.279759   
1.333250e+02 39    -31.287675  1.215618 -30.072056 -31.492468  1.276008   
1.571000e+02 40    -31.550823  1.225837 -30.324986 -31.753918  1.283187   
1.808750e+02 41    -31.783542  1.244896 -30.538646 -31.981830  1.298319   
2.046500e+02 42    -31.991941  1.270724 -30.721216 -32.183405  1.319660   
2.284250e+02 43    -32.175991  1.301368 -30.874623 -32.359529  1.345517   
2.522000e+02 44    -32.338517  1.335612 -31.002906 -32.513716  1.374803   
2.759750e+02 45    -32.482798  1.372753 -31.110045 -32.649609  1.406851   
2.997500e+02 46    -32.612391  1.412606 -31.199785 -32.770927  1.441453   
3.235250e+02 47    -32.730897  1.455488 -31.275408 -32.881277  1.478846   
3.473000e+02 48    -32.840513  1.501692 -31.338821 -32.982885  1.519243   
3.710750e+02 49    -32.942345  1.551359 -31.390987 -33.076905  1.562743   
3.948500e+02 50    -33.037811  1.604881 -31.432930 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')